# DSTC_SpGEMM

This notebook reproduces the DSTC_SpGEMM accelerator in [DSTC](https://dl.acm.org/doi/10.1109/ISCA52012.2021.00088).

## Imports

Import the necessary modules.

In [ ]:
# HiFiber boilerplate

from fibertree_bootstrap import *

fibertree_bootstrap(style="tree", animation='movie')

# Compilation boilerplate

import os
import sys
sys.path.insert(0, "..")

from src import utils

## Initialization

Initialize the input tensors. Tensor shapes and densities can be modified below.

**Warning:** Large tensors will overwhelm the video generation. Either:
1. Use small tensors; as a rule of thumb, fewer than 60 computes (e.g., multiplications) should be required.
2. Do not generate a video; remove the `spacetime` specification from the `mapping` before compiling.

In [ ]:
K = 4
M = 12
N = 8

M2 = 4
N2 = 4
M1 = 4
N1 = 4
M0 = 2
N0 = 2

density = [0.9, 0.5]
seed = 0

A_KM = Tensor.fromRandom(rank_ids=["K", "M"], shape=[K, M], seed=seed, density=density, name="A")
B_KN = Tensor.fromRandom(rank_ids=["K", "N"], shape=[K, N], seed=seed + 1, density=density, name="B")

## Compile and Run

Below is the TeAAL specification for DSTC_SpGEMM. To simulate the accelerator:
1. Compile it to HiFiber by running the cell, inserting a new cell
2. Run the new cell, which will
    - Execute the kernel; multiplying the above defined matrices
    - Generate visualizations of the actions of the kernel

#### Notes
- Small tensors are required for video generation. Partition shapes are decreased accordingly for visualization purposes. The real accelerator uses `M0 = 4`, `M1 = 8`, `N0 = 4`, and `N1 = 16`.
- If you are using large tensors, remove the spacetime specification to generate a kernel that does not produce videos. Outputs can still be checked below.

In [ ]:
yaml = """
einsum:
  declaration:
    A: [K, M]
    B: [K, N]
    Z: [M, N]
  expressions:
    - Z[m, n] = A[k, m] * B[k, n]
mapping:
  rank-order:
    A: [K, M]
    B: [K, N]
    Z: [M, N]
  partitioning:
    Z:
      M: [uniform_shape(M2), uniform_occupancy(A.M1), uniform_occupancy(A.M0)] 
      N: [uniform_shape(N2), uniform_occupancy(B.N1), uniform_occupancy(B.N0)]
  loop-order:
    Z: [M3, N3, K, M2, N2, M1, N1, M0, N0]
  spacetime:
    Z:
      space: [M2, N2, M1, N1, N0]
      time: [M3, N3, K, M0]
"""

utils.compile(yaml)

## Check Results

Check that generated code computes the correct result.

**Note**: Should be used after compiling and running the kernel (above cell).

In [ ]:
utils.check_matmul(A_KM, B_KN, Z_MN)